# Load and Use our Book Recommendation System

Here we show how to load and use our saved mode.

In [1]:
from pyspark.sql import SparkSession
from pyspark.ml.recommendation import ALSModel
from pyspark.ml.feature import IndexToString
import pyspark.sql.functions as F

spark = SparkSession.builder.appName('books').getOrCreate()


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/09/06 10:56:08 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/09/06 10:56:09 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [2]:
def Book_Recommend(user_set, num_rec):
    ### user_set: integer list
    ### num_rec: integer, number of recommendations
    
    
    model_loaded = ALSModel.load('book_recommendations')
    IndexString_model_loaded = IndexToString.load('index_to_books')
    
    user_ids = user_set
    test_user_df = spark.createDataFrame([(user_id,) for user_id in user_ids], ["User_ID"])
    loaded_rec = model_loaded.recommendForUserSubset(test_user_df, num_rec)
    exploded_loaded_rec = loaded_rec.select('User_ID', F.explode('recommendations').alias("exploded_recommendations"))
    loaded_rec_names = IndexString_model_loaded.transform(exploded_loaded_rec.select('User_ID', 'exploded_recommendations.book_index'))\
            .select('User_ID', 'Book_Title')
    
    # Get the book information
    bookpath = 'Books.csv'
    book_info = spark.read.csv(bookpath, header=True, inferSchema=True)
    
    # We modify the column names.
    for name in book_info.schema.names:
        book_info = book_info.withColumnRenamed(name, name.replace('-', '_'))
        
    book_info.createOrReplaceTempView('book_info')
    loaded_rec_names.createOrReplaceTempView('loaded_rec_names')

    query = " SELECT R.User_ID, B.ISBN, B.Book_Title, B.Book_Author, B.Year_Of_Publication, B.Publisher\
          FROM book_info B\
          INNER JOIN loaded_rec_names R\
          ON R.Book_Title = B.Book_Title"
    book_loaded_info = spark.sql(query)
    
    return book_loaded_info
    
    

In [7]:
user_ids = [507, 882]
num_rec = 5
mybooks = Book_Recommend(user_ids, num_rec)

In [8]:
mybooks.show()

+-------+----------+--------------------+--------------------+-------------------+--------------------+
|User_ID|      ISBN|          Book_Title|         Book_Author|Year_Of_Publication|           Publisher|
+-------+----------+--------------------+--------------------+-------------------+--------------------+
|    882|0060195460|Keep It Simple, S...|      Judy Sheindlin|               2000|HarperCollins Pub...|
|    882|0064472795|Princess in the S...|           Meg Cabot|               2002|        HarperTrophy|
|    882|0060294655|Princess in the S...|           Meg Cabot|               2001|       HarperCollins|
|    507|0061090565|Under Fire: An Am...|     Oliver L. North|               1992|HarperCollins Pub...|
|    507|0971700915|Under Fire: An Am...|     Oliver L. North|               1991|  21st Century Press|
|    507|0028621239|The Complete Idio...|          Gail Diven|               1999|         Alpha Books|
|    507|0060183349|Under Fire: An Am...|     Oliver L. North|  

In [9]:
user_ids = [882, 1025, 7286 ]
mybooks = Book_Recommend(user_ids, num_rec)

In [10]:
mybooks.show()

+-------+----------+--------------------+--------------------+-------------------+--------------------+
|User_ID|      ISBN|          Book_Title|         Book_Author|Year_Of_Publication|           Publisher|
+-------+----------+--------------------+--------------------+-------------------+--------------------+
|   1025|0060391901|Mastering the Zon...|         Barry Sears|               1996|          ReganBooks|
|    882|0060195460|Keep It Simple, S...|      Judy Sheindlin|               2000|HarperCollins Pub...|
|    882|0064472795|Princess in the S...|           Meg Cabot|               2002|        HarperTrophy|
|    882|0060294655|Princess in the S...|           Meg Cabot|               2001|       HarperCollins|
|   7286|0061090565|Under Fire: An Am...|     Oliver L. North|               1992|HarperCollins Pub...|
|   7286|0971700915|Under Fire: An Am...|     Oliver L. North|               1991|  21st Century Press|
|   7286|0028621239|The Complete Idio...|          Gail Diven|  